<a href="https://colab.research.google.com/github/sgr1118/PyTorch/blob/main/Chapter15_%EC%8B%A4%EB%AC%B4_%ED%99%98%EA%B2%BD%EC%97%90%EC%84%9C%EC%9D%98_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 15.1 실무를 진행하듯 실습하기
- 주피터 노트북은 데이터를 EDA과 같이 각 셀의 결과에 따라 해야 하는 일이 바뀌는 경우에 적합하다. 하지만 작업이 명확하고 반복적이라면 py 확장자를 가진 파이썬 스크립트로 제작하는 CLI 환경에서 작업을 수행하는 것이 더 바람직하다.

### 1. ML 프로젝트 파일 구조 예시

 |파일명|설명|
 |-|-|
 |model.py|모델 클래스가 정의된 코드|
 |trainer.py|데이터를 받아와 모델 객채를 학습하기 위한 트레이너가 정의된 코드|
 |dataloader.py|데이터 파일을 읽어와 전처리를 수행하고 신경망에 넣기 좋은 형태로 변환하는 코드|
 |train.py|사용자로부터 하이퍼퍼라미터를 입력받아 필요한 객체들을 준비하여 학습을 진행|
 |predict.py|사용자로부터 기학습된 모델과 추론을 위한 샘플을 입력받아 추론을 수행|

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/01-overview.png)
<center>ML프로젝트 파일 간의 상호작용</center>

- 사용자는 train.py를 통하여 코드 수정 없이 다양한 하이퍼파라미터들을 변경해가며 반복적인 실험을 수행

- CLI 환경에서 train.py를 실행하고 하이퍼파라미터를 아규먼트를 통해 전달한다. 아래의 실행 명령은 모델 가중치 파일이 저장될 파일 경로와 모델의 깊이 그리고 드롭아웃의 확률을 train.py에 넘겨준다

- $ python train.py --model_fn ./models/model.pth --n_layers 10 --dropout 0.3

- 또한 트레이너는 데이터로더로 부터 준비된 데이터를 넘겨받아 모델에 넣어 학습과 검증을 진행한다. 학습이 완료되면 모델의 가중치 파라미터는 보통 피클 형태로 다른 필요한 정보(모델을 생성하기 위한 각종 설정 및 하이퍼파라미터)들 과 함께 파일로 저장된다.

- predict.py는 저장된 피클을 읽어와서 모델의 객체를 생성하고 학습된 가중치 파라미터를 그대로 복원한다. 그리고 사용자로부터 추론을 위한 샘플이 주어지면 모델에 통과시켜 추론값을 반환한다. 이때 predict.py에 선언된 함수들을 감싸서 RESTful API 서버로 구현할 수도 있을 것이다.

## 15.2 워크플로 리뷰

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/02-workflow.png)
<center>ML 프로젝트 워크플로</center>


### 1. 문제 정의
- 손글씨 숫자를 인식하는 함수 $f^*$을 근사 계산할 것이다. 이 모델을 만들기 위해 손글씨 데이터를 수집하고 이에 대한 레이블링도 수행한다.

### 2. 데이터 수집
- 실무 환경에서는 데이터가 없거나 존재하더라고 레이블링이 되지 않은 경우를 맞이하게된다. 수집과 레이블링 작업을 수행하기 위한 두 가지 선택지가 있다. 첫 번째 직접 수행하기 두 번째 외주 작업 요청이다.

- 데이터를 준비할 때 예상 소요시간을 잘 계산하는 것이 좋다. 무턱대고 예산을 받아 데이터를 수집하고 모델을 만드는 것은 프로젝트가 잘 진행되지 않았을 때 매우 위험하다. 따라서 개념 증명 과정을 거쳐서 실현 가능성을 확인해야 한다. 이때 프로토타입 모델에 일부 데이터만을 학습시켜 실현 가능성을 보여주면 된다.

### 3. 데이터 전처리
- 수집한 데이터를 학습/검증/추론용으로 분할하고 전처리를 수행한다. 이때 데이터의 성격에 따라 전처리가 매우 다르기 때문에 데이터가 어떤 분포와 형태를 띄고있는지 자세히 분석해야한다. 아래 그림을 참고하자.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/02-preproc.png)
<center>데이터 종류와 형태 등에 따른 다양한 전처리 기법</center>

- 일부 전처리 기법들은 데이터를 기반으로 파라미터가 결정된다. 앞서 배웠듯이 정규화 스케일은 학습 데이터셋 기준으로 수행하여야 한다. 전체 데이터셋을 기준으로 할 경우 테스트셋을 보고 테스트를 평가하는 것과 다를바가 없기 때문이다.

### 4. 알고리즘 적용
- 전처리 과정에서 수행된 분석을 통하여 분포나 성징을 파악하였을 것이다. 이 분석 결과를 바탕으로 알맞은 가설을 설정하고 알고리즘 구현 및 적용을 하여여한다. 물론 이 과정에서 반드시 DNN을 사용해야만하는 것은 아니며 적절한 ML 알고리즘을 적용하면 된다. 만약 DNN을 적용하기로 결정했다면 아래 과정을 참고하자.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/02-how_to_set_architecture.png)
<center>신경망 구조 결정 과정</center>

- 계층의 개수, 활성 함수의 종류, 정규화 방법 등의 하이퍼파라미터도 결정해야하는데 아래 그림을 참고하자.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/02-how_to_set_architecture2.png)
<center>신경망 구조 정의 과정</center>

- 적당한 초기 하이퍼파라미터를 설정하여 오버피팅이 될 때까지 신경망을 깊고 넓게 만든다. 오버피팅이 발생한 것을 확인했다면 데이터셋의 복잡한 데이터를 신경망이 충분히 학습할 만한 수용능력을 가지고있음을 보여준다. 또한 오버피팅이 발생하더라도 매 에포크마다 검증 데이터셋에 대한 손실 값을 추적하고 있으므로 큰 문제가 되지않는다. 이후 적절한 score metric을 적용하여 평가하고 성능을 수치화한다. 여기까지 과정이 모델링 과정을 거친 것이 되고 이후 하이퍼파라미터를 수정하며 이 과정을 반복하여 모델의 성능을 개선시킨다.

- 하이퍼파라미터 수정으로 성능 개선이 이루어지지않는다면 원인에 대한 가설을 설정하고 모델의 구조를 바꾸는 등 수정을 거쳐야한다.

### 5. 평가
- 평과 결과에 따라 언더피팅이 의심될 경우에는 모델의 수용 능력을 올리는 방향으로 하이퍼파라미터를 튜닝하고 오버피팅으로인하여 일반화 성능이 저하되는 것이 우려라면 정규화 기법을 강화하는 방향으로 튜닝하면서 학습과 평가를 반복 수행하게 될 것이다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/02-tuning.png)
<center>모델 평가 결과에 개선 작업</center>

|범주|학습 데이터셋|검증 데이터셋|테스트 데이터셋|
|-|-|-|-|
|가중치 파라미터|결정|검증|검증|
|하이퍼파라미터||결정|검증|
|알고리즘|||결정|

### 6. 배포
- 이번 실습에서는 배포를 위한 추론 코드는 손쉬운 시각화를 위해서 노트북을 통해 구현되었다.

## 15.3 실습 소개
- 데이터 증강은 데이터의 핵심 특징을 간직한 채 노이즈를 더하여 데이터셋을 확장하는 방법이다.

### 1. 모델 구조 설계
- 아래 그림은 이번에 설계할 모델의 구조이다.

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/03-model_architecture.png)
<center>MNIST 분류기 구현 모델의 구조</center>

- 하나의 블록 내에는 선형 계층과 비선형 활성 함수인 리키렐루 그리고 정규화를 위한 배치정규화 계층이 차례대로 들어간다. 필요에 따라 드롭아웃을 넣을수도있다.

- 이렇게하면 모델은 각 클래스별 로그 확률 값을 출력하고 이것을 정답을 위한 원-핫 벡터와 비교하면 손실값을 계산할 수 있다. 이때 그냥 소프트맥스 함수가 아닌 로그소프트맥스 함수를 사용하기 때문에 NLL 손실 함수를 사용한다.

### 2. 학습 과정
- DNN 학습 과정은 아래와 같다

![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-15/03-training_procedure.png)
<center>DNN을 학습하기 위한 과정</center>

##### 학습 과정을 수식으로 표현
- 앞서 그림의 모델 학습 과정의 수식은 다음과 같다. 우리는 N개의 입출력 쌍을 모아 데이터셋을 구축한다.

- $\mathcal{D} = {(x_i, y_i)}_{i=1}^N$
- $where x \in [0,1]^N*(28*28) and y \in {0,1}^N*10$

- 이때 입력 샘플의 경우 28*28 크기의 이미지를 변환하여 784차원의 벡터로 만든다. 또한 정답 출력 벡터는 10차원의 원-핫 벡터이다.

- 가중치 파라미터 $\theta$를 갖는 모델 $f_{\theta}$를 통해 $f^*$를 근사 계산하고자합니다. 이때 이 문제를 확률 문제로 접근할 수 있다. 따라서 모델 $f_{\theta}$의 출력 벡터 $\hat{y}$은 각 클래스별 로그 확률 값이 담겨있는 벡터라고 볼 수 있는데 로그소프트맥스 함수를 활용하여 구현 가능하다.

- $logP_{\theta}(\cdot|x_i) = \hat{y}_i = f_{\theta}(x_i)$

-  이제 손실 함수를 정의해 볼 수 있다. 손실 함수는 NLL 손실 함수를 사용한다. 파이토치에서 쓰이는 nn.NLLLoss 클래스를 사용하면 된다.

- 이렇게 정의된 손실 함수를 최소화하는 입력 파라미터를 찾게 되면 우리가 원하는 $f^*$를 잘 근사하는 가중치 파라미터가 될 것이다. 이러한 작업을 수행하기 위해 경사하강법을 통해 파라미터를 업데이트하여 손실 값을 최소화하는 가중치 파라미터를 점진적으로 찾을 수 있다. (이 과정 수식은 앞장에서 다뤘기 때문에 생략)

### 3. 파일 구조
- 아래와 같은 구조로 py를 만들 것이다. 지금은 소규모 프로젝트이기 때문에 이 파일을 전부 한 디렉토리 내에 위치하게 할 것이다. 나중에 규모가 커지면 디렉토리 구조를 추가할 필요가있다. 예를 들어 다른 신경망 구조를 사용하게 되는 경우

 |파일명|설명|
 |-|-|
 |model.py|모델 클래스가 정의된 코드|
 |trainer.py|데이터를 받아와 모델 객채를 학습하기 위한 트레이너가 정의된 코드|
 |dataloader.py|데이터 파일을 읽어와 전처리를 수행하고 신경망에 넣기 좋은 형태로 변환하는 코드|
 |train.py|사용자로부터 하이퍼퍼라미터를 입력받아 필요한 객체들을 준비하여 학습을 진행|
 |predict.py|사용자로부터 기학습된 모델과 추론을 위한 샘플을 입력받아 추론을 수행|

### 4. 실습 파일 위치
- 자신이 편한 위치에 임의로 생성을 하자




## 15.4 ~ 8은 따로 py파일을 만들기 때문에 py에 설명을 따로 확인하자

## 15.9 마치며

### 요약

1. 프로젝트가 반영해야 할 요구사항
 - 효율적으로 실험을 반복해서 수행할 수 있어야 한다.
 - 모델 아키텍처가 바뀌어도 바로 동작할 수 있어야 한다.
 - 하이퍼파라미터를 바꿔서 다양한 실험을 돌릴 수 있어야한다.
 - 코드의 일부분이 수정되어도 다른 부분은 큰 수정이 없도록 독립적으로 동작해야 한다.

2. 개선 사항

|AS-IS|TO-BE|
|-|-|
|데이터 전처리/분할 등을 직접 구현|Pytorch Dataset을 활용|
|여전치 custom for-loop에 의존|PyTorch lgnite/Lightning과 같은 라이브러리 활용|